In [1]:
import numpy as np
import os.path as osp
from rrc.envs import cube_env, initializers
from gym.wrappers import FlattenObservation, FilterObservation

from spinup.utils import test_policy
from spinup.user_config import DEFAULT_DATA_DIR
from xvfbwrapper import Xvfb
from rrc.envs.wrappers import PyBulletClearGUIWrapper

/scr1/.pyenv/versions/miniconda3-latest/envs/rrc/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
# env, pol = test_policy.load_policy_and_env('/scr-ssd/ksrini/spinningup/data/2021-06-03_ppo-rrc-acclip/2021-06-03_10-59-54-ppo-rrc-acclip_s0')
env, pol = test_policy.load_policy_and_env('/scr-ssd/ksrini/spinningup/data/2021-06-03_ppo-rrc-diff4_irandom/2021-06-03_12-03-09-ppo-rrc-diff4_irandom_s0')



Loading from /scr-ssd/ksrini/spinningup/data/2021-06-03_ppo-rrc-diff4_irandom/2021-06-03_12-03-09-ppo-rrc-diff4_irandom_s0/pyt_save/model499.pt.




In [4]:
xvfb = Xvfb()
xvfb.start()

goal_pose = None # {'position': np.array([0,0,0.0325]), 'orientation': np.array([0,0,0,1])}
env = cube_env.CubeEnv(goal_pose, 1, initializer=env.initializer, episode_length=500,
                       visualization=True, save_mp4=True, save_dir='./videos')
env = PyBulletClearGUIWrapper(env)
env = FlattenObservation(FilterObservation(env, filter_keys=['observation', 'desired_goal']))
obs = env.reset()
d = False
while not d:
    obs, r, d, i = env.step(pol(obs))
obs = env.reset()
xvfb.stop()

/scr1/.pyenv/versions/miniconda3-latest/envs/rrc/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [9]:
obs, r, d, i = env.step(pol(obs))

(array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  3.2499999e-02, -9.8478017e-07,
        -1.3081907e-05, -5.7593757e-01,  8.1749368e-01,  6.4295933e-02,
         7.0091151e-02,  1.3454532e-02, -6.0875551e-05,  4.8068559e-05,
         4.1382122e-01, -1.4876003e-03, -2.2157433e-03, -1.1878055e-05],
       dtype=float32),
 -0.21716879579162512,
 False,
 {'difficulty': 1})

In [ ]:
xvfb = Xvfb()
xvfb.start()

goal_pose = {'position': np.array([0,0,0.0325]), 'orientation': np.array([0,0,0,1])}
env = cube_env.CubeEnv(goal_pose, 1, initializer=initializers.CenteredInitializer(1), episode_length=500,
                       visualization=True, save_mp4=True, save_dir='./videos')
env = FlattenObservation(FilterObservation(env, filter_keys=['observation', 'desired_goal']))
obs = env.reset()
d = False
while not d:
    obs, r, d, i = env.step(pol(obs))

/scr1/.pyenv/versions/miniconda3-latest/envs/rrc/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
goal_pose = {'position': np.array([0,0,0.0325]), 'orientation': np.array([0,0,0,1])}
env = cube_env.CubeEnv(goal_pose, 1, initializer=initializers.CenteredInitializer(1), episode_length=1000)
env = FlattenObservation(FilterObservation(env, filter_keys=['observation', 'desired_goal']))

/scr1/.pyenv/versions/miniconda3-latest/envs/rrc/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
obs = env.reset()

In [12]:
d = False
while not d:
    o,r,d,i = env.step(env.action_space.sample()*.15)

In [13]:
env.reset()

array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  3.2499999e-02,  1.4490881e-06,
       -1.6389138e-06,  7.1549618e-01,  6.9861662e-01, -5.0409850e-02,
       -3.3456851e-02,  1.1799225e-02, -8.4705953e-06, -3.2415748e-05,
        4.4980615e-01,  1.0924968e-03, -5.4077373e-05, -1.4260825e-04],
      dtype=float32)